# TD twitter bot classification

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from pycaret.classification import (
    setup, compare_models, plot_model, blend_models,
    tune_model, save_model, load_model, finalize_model)
from pycaret.regression import interpret_model

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'DejaVu Sans'

# I. Chargement des données
Charger le csv `twitter_human_bots_dataset.csv` dans un dataframe que vous nommerez `df`

In [ ]:
df = pd.read_csv('../data/twitter_human_bots_dataset_light.csv')
print(f"Dataset shape {df.shape}")
df.head(5)

# II. Statistiques descriptives

**Afficher le nombre de bots et le nombre d'humains (voir la colonne `account_type`)**

In [ ]:
df['account_type'].value_counts()

**Afin de mettre en lien la "target" (`account_type`) et la variable `verified`, afficher un bar plot en affichant une couleur différente en fonction de la colonne `verified`**

Indice : `account_type` en x, le nombre d'occurence par "target" en y

In [ ]:
df_plot = df[["account_type", "verified"]].groupby(["account_type", "verified"])["account_type"].count().unstack("verified")
df_plot.plot(kind='bar', stacked=True)

In [ ]:
import pygwalker as pyg
walker = pyg.walk(df)

### Facets Dive

Dive est un outil permettant d'explorer de manière interactive un grand nombre de points de données à la fois. Il fournit une interface interactive pour explorer la relation entre les points de données à travers toutes les différentes caractéristiques d'un ensemble de données. Chaque élément individuel de la visualisation représente un point de données. Positionnez les éléments en les "facettant" ou en les classant dans plusieurs dimensions en fonction des valeurs de leurs caractéristiques.

Demo : https://pair-code.github.io/facets/quickdraw.html

In [ ]:
df_sample = df.sample(10000)

In [ ]:
# Display the Dive visualization for the training data.
from IPython.display import display, HTML

jsonstr = df_sample.to_json(orient='records')
HTML_TEMPLATE = """
        <script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
        <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html">
        <facets-dive id="elem" height="600"></facets-dive>
        <script>
          var data = {jsonstr};
          document.querySelector("#elem").data = data;
        </script>"""
html = HTML_TEMPLATE.format(jsonstr=jsonstr)
display(HTML(html))

# III. Classifier de bot

## a. Features engineering
Un modèle de Machine Learning ne peut prendre en entrée que des entiers ou des réels. L'objectif de cette partie est de transformer intelligement les données brutes en variables (features en anglais) intelligibles pour notre modele.

In [ ]:
df.location.value_counts()[:20]

In [ ]:
df.iloc[0]

**Créer une variable `X` qui est une copie de `df`. C'est maintenant sur cette variable `X` que nous allons travailler**

Indice : utiliser `.copy()`

In [ ]:
X = df.copy()
X.head()

**Convertisser les variables `"default_profile", "default_profile_image", "geo_enabled", "verified"` en entier**

Indice : utiliser `.astype(int)`

In [ ]:
# Preprocess boolean columns
for col in ["default_profile", "default_profile_image", "geo_enabled", "verified"]:
    X[col] = X[col].astype(int)
X.head()

**Créér une nouvelle colonne que vous nommerez `popularity` en appelant la fonction `compute_popularity_metric` qui ajoute un score de "popularité" en fonction du nombre d'amis et du nombre de followers**

In [ ]:
def compute_popularity_metric(row):
    return np.round(np.log(1+row["friends_count"]) * np.log(1+row["followers_count"]), 3)

# Create a custom metric to measure the popularity of an input account
X["popularity"] = X.apply(compute_popularity_metric, axis=1)
X['popularity']

In [ ]:
import plotly.express as px
fig = px.histogram(X, x="popularity", color='account_type')
fig.update_layout(barmode='overlay') # 'stack', 'group', 'overlay', 'relative'
fig.show()

In [ ]:
# Let's show some examples of such value
#X[['popularity']].hist(bins=100)

In [ ]:
df_plot = X.copy()
df_plot['popularity_bins'] = pd.cut(
    df_plot['popularity']/df_plot['popularity'].max()*100,
    bins=list(range(0, 110, 20)))
# plot
(df_plot.groupby(["account_type", 'popularity_bins'])["account_type"]
    .count()
    .unstack('popularity_bins')
    .plot(kind='bar', stacked=True))

**Convertir la colonne target en catégorie**

Indices :
- vous devez caster la colonne `account_type` en `category` avec `.astype('category')`
- écraser ensuite les valeurs de la colonne avec `.cat.codes` pour récupérer un entier

In [ ]:
"""
# Methode 1 :
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(X["account_type"])
le.transform(X["account_type"])
"""

# Method 2 :
X["account_type"] = X["account_type"].astype('category').cat.codes
X["account_type"]

## b. Features selection

**Ne garder maintenant que les valeurs numériques**

Indice : utiliser `_get_numeric_data()`

In [ ]:
X = X._get_numeric_data()
X.head()

**Supprimer la colonne `id`, elle ne porte aucune information pour le modele de classification**

Indice : utiliser `drop(..., axis=1,inplace=True)`

In [ ]:
X.drop(columns=["id"], inplace=True, errors='ignore')
print(X.shape)
X.head()

**Calculer la matrice de correlations des variables**

Indice : utiliser `.corr()`

In [ ]:
# Compute correlation among the features and the response variable
corr = X.corr()

**Afficher cette matrice de correlations**

In [ ]:
sns.heatmap(corr,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,
            annot=True, fmt='.1g', cmap='coolwarm')

In [ ]:
X.to_csv('X.csv', index=None)

## c. AutoML

**Séparer votre dataset `X` en deux sets de données via la méthode `train_test_split`. Utiliser `0.2` (=20% du dataset) pour la proportion du `test_size`**

In [ ]:
# voir le discord pour reprendre ici, vous aurez juste besoin du X.csv
X = pd.read_csv('X.csv')
X.shape

In [ ]:
X = X.sample(20000, random_state=42)

In [ ]:
dataset_train, dataset_unseen = train_test_split(X,
                                                 test_size=0.2,
                                                 random_state=42,
                                                 stratify=X['account_type'])
dataset_train.shape, dataset_unseen.shape

**Il est maintenant temps de préparer la classification avec l'appel à la fonction `setup` de PyCaret !**

In [ ]:
setup(data=dataset_train,
      test_data=dataset_unseen,
      target="account_type",
      session_id=42,
      fold=3)

**Passez au crible les modèles de PyCaret et comparez leurs performances.**

Quel est le modèle avec la meilleure exactitude ?

Indice : utiliser `compare_models` de PyCaret

In [ ]:
best_model = compare_models()

In [ ]:
best_model = compare_models()

<b>Réponse :</b>

Le meilleur modèle est ici un `CatBoost Classifier`. On estime sur la base de la validation k-fold qu'il possède 87 % d'exactitude dans ses prédictions.

Il offre par ailleurs des taux de rappel et de précision relativement équilibrés.

En regardant les performances des autres classifieurs, on note au passage :

- plusieurs classifieurs proposent des performances relativement similaires en haut du classement, ce qui semble indiquer une convergence globale vers un optimum. Il est probable que varier la formulation du classifieur ne résultera que dans des gains marginaux de performance.

- on observe globalement le compromis Precision/Recall dans les scores des classifieurs : l'augmentation de l'un correspond à la diminution de l'autre. En effet, chaque classifieur a sa propre façon de fixer le seuil d'acceptabilité d'une prédiction en tant que "Vrai". Faire varier ce seuil revient à troquer des Faux Positifs contre des Faux Négatifs.

**Affichez la matrice de confusion sur les données de test.**

Que pouvez-vous en dire ?

Indice : utiliser `plot_model` de PyCaret

In [ ]:
best_model

In [ ]:
plot_model(best_model, plot='confusion_matrix')

Sur la base des nombres de cette matrice, calculez la précision et le rappel dans l'échantillon de test.

Observez-vous les mêmes performances que dans la validation k-fold ?

In [ ]:
# Vrai Positif = un individu identifié comme bot l'est effectivement.
# C'est-à-dire : True Class == 1 et Predicted Class == 1
VP = 1907

# Faux Positif = un individu est identifié abusivement comme un bot.
# C'est-à-dire : True Class == 0 et Predicted Class == 1
FP = 578

# Faux Négatif = un individu est identifié abusivement comme un utilisateur légitime.
# C'est-à-dire : True Class == 1 et Predicted Class == 0
FN = 316

precision = VP / (VP + FP)
recall = VP / (VP + FN)

print(f"{precision} et {recall}")

<b>Le modèle généralise bien à l'échantillon de test</b>, car il délivre des performance quasiment similaires à celles obtenues en train (~ 71% de précision et de rappel en test, contre 72% en train)

## d. Pour aller plus loin

Pour améliorer les performances, il est possible de faire varier les hyper-paramètres du modèle.

Cela peut se faire à l'aide d'un simple appel à la fonction `tune_model`, [décrite ici](https://pycaret.org/tune-model/).

<b>Reprenez le meilleur modèle et optimisez ses hyper-paramètres.</b> Constatez-vous une amélioration des performances ?

In [ ]:
best_model_tuned = tune_model(best_model)

In [ ]:
best_model_tuned

On note une stagnation des performances, avec presque la même matrice de confusion.

Une autre piste d'amélioration réside dans la **combinaison de plusieurs modèles**, et le **blending** de leurs prédictions via un vote majoritaire.

Cela peut se faire à l'aide d'un simple appel à la fonction `blend_models`, [décrite ici](https://pycaret.org/blend_models/).

Reprenez les 5 meilleur modèles et effectuez un blending. Constatez-vous une amélioration des performances en test ?

In [ ]:
%%time
top_5_models = compare_models(n_select=5)

blended_model = blend_models(top_5_models)

In [ ]:
plot_model(blended_model, plot='confusion_matrix')

<b>Ici aussi, on note une stagnation des performances, avec presque la même matrice de confusion.</b>

Pour conclure ce TP, on se propose de fournir une explication <i>a posteriori</i> du fonctionnement du classifieur.

Pour obtenir le graphique d'importance des variables explicatives, appelez la fonction `plot_model` avec comme argument le modèle de votre choix et avec l'argument `plot="feature"`.

Comment interprétez-vous ces valeurs ?

In [ ]:
plot_model(best_model,'feature')

**Afficher la courbe ROC**

Indice : utiliser `plot_model` avec  `plot='auc'`

In [ ]:
plot_model(best_model, plot='auc')

**Afficher la courbe Précision-Recall**

Indice : utiliser `plot_model` avec  `plot='pr'`

In [ ]:
plot_model(best_model_tuned, plot='pr')

In [ ]:
plot_model(best_model_tuned, plot='threshold')

**Afficher l'explicabilité**

Indice : utiliser `interpret_model`

In [ ]:
interpret_model(best_model)

**Entrainer le modele final**

Indice : utiliser `finalize_model` en réutilisant le best model tuned

In [ ]:
final_model = finalize_model(best_model_tuned)

In [ ]:
final_model

**Sauvegarder le modele final**

Indice : utiliser `save_model`

In [ ]:
save_model(final_model, 'ml_model')

**Recharger le modele final**

Indice : utiliser `load_model`

In [ ]:
load_model('ml_model')